In [1]:
from pinecone import Pinecone, ServerlessSpec
import itertools
from tqdm import tqdm
import json

# Environment variables

In [4]:
pinecone_api_key = ""   # TODO: Pinecone api key
pinecone_index_name = ""    # TODO: Name of the pinecone index where the vectors will be stored
json_filepath = ""      # TODO: Path to the json file from '03_Embedding.ipynb'

# Create new index (if none exists)

In [5]:
pc = Pinecone(api_key=pinecone_api_key)

pc.create_index(
    name=pinecone_index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# Setup index

In [6]:
index = pc.Index(pinecone_index_name)

# Load json data from '03_Embedding.ipynb'

In [7]:
with open(json_filepath, "r") as file:
    vectors = json.load(file)

# Upsert vectors to pinecone

In [8]:
def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [9]:
for vec_chunks in tqdm(chunks(vectors, batch_size=100)):
    index.upsert(vectors=vec_chunks)

101it [00:35,  2.88it/s]
